# Importing libraries 

In [7]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [13]:
True_news=pd.read_csv('True.csv')
Fake_news=pd.read_csv('Fake.csv')

In [17]:
True_news['label']=0
Fake_news['label']=1

In [19]:
data1=True_news[['text','label']]
data2=Fake_news[['text','label']]

In [21]:
dataset=pd.concat([data1,data2])

In [23]:
dataset.shape

(44898, 2)

In [96]:
dataset.isnull().sum()

text     0
label    0
dtype: int64

In [97]:
dataset['label'].value_counts()

label
1    23481
0    21417
Name: count, dtype: int64

# Shuffle data

In [25]:
dataset=dataset.sample(frac=1)

# NLP process

In [29]:
ps=WordNetLemmatizer()
stopwords=stopwords.words('english')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Cleaning text 

In [31]:
def clean_row(row):
    row=row.lower()
    row=re.sub('[^a-zA-Z]',' ',row)

    tokens=row.split()
    news=[ps.lemmatize(word) for word in tokens if not word in stopwords]
    cleaned_news = ' '.join(news)
    return cleaned_news

dataset['text']=dataset['text'].apply(lambda x : clean_row(x))

In [33]:
vectorize=TfidfVectorizer(max_features = 5000 ,lowercase=False,ngram_range=(1,2))

In [35]:
x=dataset.iloc[:30000,0]
y=dataset.iloc[:30000,1]

In [37]:
train_data,test_data,train_label,test_label=train_test_split(x,y,test_size=0.2,random_state=0)

# text vectorizing

In [39]:
vec_train_data=vectorize.fit_transform(train_data)
vec_train_data=vec_train_data.toarray()

In [40]:
vec_test_data=vectorize.fit_transform(test_data)
vec_test_data=vec_test_data.toarray()

In [41]:
training_data=pd.DataFrame(vec_train_data,columns= vectorize.get_feature_names_out())
testing_data=pd.DataFrame(vec_test_data,columns= vectorize.get_feature_names_out())

# Model 

In [42]:
clf=MultinomialNB()
clf.fit(training_data,train_label)


MultinomialNB()

In [43]:
y_pred=clf.predict(testing_data)
#accuracy_score()

In [44]:
y_pred_train=clf.predict(training_data)
accuracy_score(train_label,y_pred_train)

0.945625

In [53]:
txt="Pushpa 2 Box Office Collection Day 14: No Mid Week Blues, Allu Arjun-Starrer Is Just Rs 42 Crores Away To Break Baahubali 2's Record"
news=clean_row(txt)
pred=clf.predict(vectorize.transform([news]).toarray())
if pred==0:
    print('True news')
else:
    print('Fake news')

True news


C:\Users\hp\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(
